In [2]:
# !conda install -c conda-forge python-dotenv -y
import numpy as np
import pandas as pd
from deap import base, creator, tools
from functools import reduce
from dotenv import load_dotenv
import finlab
import os
import plotly.express as px
from finlab import data
from finlab.backtest import sim
from finlab.dataframe import FinlabDataFrame
from finlab.portfolio import Portfolio
from finlab.plot import plot_tw_stock_candles
from finlab.data import indicator

# 載入.env檔案中的環境變數

load_dotenv()
# 這裡要替換成自己的FinLab VIP token
token = os.environ.get("FINLAB_TOKEN")
finlab.login(token)



輸入成功!


## ATR波動率的定義
使用還原全息股價，計算 ATR/收盤價。   
使用 ATR 的好處在除了最高最低價，還會會考慮 K 棒跳空因素，會比較符合波動真實情況。  
接著再取每日市場前 20% 波動率高的標地，計算市場上波動大的股票當市場指標，排掉波動過小的標地。 最後取指標的5日均線做平滑。

In [3]:
adj_close = data.get('etl:adj_close')
adj_high = data.get('etl:adj_high')
adj_low = data.get('etl:adj_low')

amplitude = abs(adj_high - adj_low)/(adj_close.shift())
amplitude_ind = amplitude[amplitude > amplitude.quantile_row(0.8)].mean(axis=1).rolling(5).mean()
amplitude_ind

date
2007-04-23         NaN
2007-04-24         NaN
2007-04-25         NaN
2007-04-26         NaN
2007-04-27         NaN
                ...   
2025-03-10    0.052549
2025-03-11    0.052771
2025-03-12    0.053859
2025-03-13    0.055854
2025-03-14    0.055547
Length: 4401, dtype: float64

In [4]:
atr = data.indicator('ATR', adjust_price=True,timeperiod=10)
adj_close = data.get('etl:adj_close')
entry_volatility = atr/adj_close
entry_volatility

,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2007-04-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-10,NaN,0.014476,0.010904,0.016175,0.014461,NaN,0.007105,0.008354,0.011931,NaN,...,0.009172,0.012526,0.028858,0.035362,0.016262,0.016725,0.029167,0.037196,0.009602,0.019995
2025-03-11,NaN,0.016640,0.013178,0.018875,0.017084,NaN,0.008659,0.009700,0.014490,NaN,...,0.009250,0.013521,0.027658,0.034570,0.017424,0.017804,0.029352,0.045017,0.010012,0.022539
2025-03-12,NaN,0.016442,0.012350,0.018689,0.016283,NaN,0.008071,0.009200,0.014109,NaN,...,0.009320,0.015130,0.026310,0.033800,0.017888,0.016553,0.031757,0.044650,0.010147,0.021356
